In [1]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')
import os
import pandas as pd
import numpy as np
# import seaborn as sns
from utilities import* 
from pandas import read_excel
from matplotlib import pyplot as plt
plt.rc('pdf',fonttype = 42)
plt.rc('ps',fonttype = 42)
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage,AnchoredOffsetbox
from matplotlib import rc, rcParams
rc('font', weight='bold')
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pymongo
from pymongo import MongoClient
from pymongo import InsertOne, DeleteMany, ReplaceOne, UpdateOne
from os import listdir
from os.path import isfile, join
import datetime
from pprint import pprint

In [8]:
def create_algo_detail(db):
    LazyCBS = {
        "algo_name": "Lazy-CBS",
        "authors": "Gange et al.",
        "papers": "Graeme Gange, Daniel Harabor and Peter J. Stuckey. Lazy CBS: Implicit Conflict-Based Search Using Lazy Clause Generation. In Proceedings of the Twenty-Ninth International Conference on Automated Planning and Scheduling 2018.",
        "github": "https://bitbucket.org/gkgange/lazycbs/src/master/",
        "comments": "",
        'user_id':""
    }
    db['algorithms'].insert_one(LazyCBS)
    
def get_path(filename,number_of_agents):
    with open(filename) as file:
        planString = ""
        number_of_line = 0
        for line in file:
            number_of_line = number_of_line + 1
            planString  = planString + line
        planString = planString.rstrip("\n")
        if(number_of_agents != number_of_line):
            print("ohhhhhhhh fuck!!!!!!!!!!!!!!!!!!")
            print("ohhhhhhhh fuck!!!!!!!!!!!!!!!!!!")
            print(number_of_agents)
            print(number_of_line)
            print("Agents number not matched")
    return planString

def insert_submission(db,submission,instance_id,algo_name,path):
    current_instance = db['instances'].find_one({'_id':instance_id})
    path_id = None
    if(submission['solution_cost']!= None):
        if(current_instance['lower_cost'] != None):
            if(current_instance['lower_cost'] > submission['solution_cost']):
                print("Inserting error: existing lower cost larger than submission solution cost")
#         //insert solution
        if(current_instance['solution_cost'] == None):
            submission["best_solution"]= True
            db['instances'].update_one({"_id" : instance_id},{"$set": 
                                                              {"solution_cost": submission['solution_cost'],
                                                                "solution_date":submission['date'],
                                                                "solution_path": path,
                                                                "empty": False
                                                              },
                                                              "$push":
                                                              {
                                                                 "solution_algos":{
                                                                     "algo_name": algo_name,
                                                                     "algo_id": submission['algo_id'],
                                                                     "date":submission['date'],
                                                                 }
                                                              }
                                                             })
        else:
            if(current_instance['solution_cost'] > submission['solution_cost']):
                db['submissions'].update_many({"instance_id" :  instance_id},{"$set": 
                                                      { 
                                                          "best_solution" : False
                                                      }
                                                     })
                submission["best_solution"]= True
                db['instances'].update_one({"_id" : instance_id},{"$set": 
                                                                    {
                                                                "solution_cost": submission['solution_cost'],
                                                                "solution_algos":[{
                                                                     "algo_name": algo_name,
                                                                     "algo_id": submission['algo_id'],
                                                                     "date":submission['date'],
                                                                    }],
                                                                "solution_date": submission['date'],
                                                                "solution_path": path,
                                                                "empty": False
                                                                    }
                                                             })
                
            elif (current_instance['solution_cost'] == submission['solution_cost']):
                submission["best_solution"]= True
                solution_inserted = False
                for algo in current_instance["solution_algos"]:
                    if(algo['algo_name'] == algo_name):
                        solution_inserted = True
                        
                if(solution_inserted == False):
                    db['instances'].update_one({"_id" : instance_id},{
                                                      "$push":
                                                      {
                                                         "solution_algos":{
                                                             "algo_name": algo_name,
                                                             "algo_id": submission['algo_id'],
                                                             "date":submission['date'],
                                                         }
                                                      }
                                                     })   
          
    if(submission['lower_cost'] != None):
        if(current_instance['solution_cost'] != None):
            if(submission['lower_cost'] > current_instance['solution_cost']):
                print("Inserting error: submission lower cost larger than existing solution cost")
        if(current_instance['lower_cost'] == None):
            submission["best_lower"]= True
            db['instances'].update_one({"_id" : instance_id},{"$set": 
                                                              {"lower_cost": submission['lower_cost'],
                                                                "lower_date":submission['date'],
                                                                "empty": False
                                                              },
                                                              "$push":
                                                              {
                                                                 "lower_algos":{
                                                                     "algo_name": algo_name,
                                                                     "algo_id": submission['algo_id'],
                                                                     "date":submission['date'],
                                                                 }
                                                              }
                                                             })
        
        else:
            if(current_instance['lower_cost'] < submission['lower_cost']):
                db['submissions'].update_many({"instance_id" :  instance_id},{"$set": 
                                                      { 
                                                          "best_lower" : False
                                                      }
                                                     })
                submission["best_lower"]= True
                db['instances'].update_one({"_id" : instance_id},{"$set": 
                                                                    {
                                                                "lower_cost": submission['lower_cost'],
                                                                "lower_algos":[{
                                                                     "algo_name": algo_name,
                                                                     "algo_id": submission['algo_id'],
                                                                     "date":submission['date'],
                                                                    }],
                                                                "lower_date": submission['date'],
                                                                "empty": False
                                                                    }
                                                             })
            elif (current_instance['lower_cost'] == submission['lower_cost']):
                submission["best_lower"]= True
                lower_inserted = False
                for algo in current_instance["lower_algos"]:
                    if(algo['algo_name'] == algo_name):
                        lower_inserted = True
                if(lower_inserted == False):
                    db['instances'].update_one({"_id" : instance_id},{
                                                      "$push":
                                                      {
                                                         "lower_algos":{
                                                             "algo_name": algo_name,
                                                             "algo_id": submission['algo_id'],
                                                             "date":submission['date'],
                                                         }
                                                      }
                                                     })  
        
    current_instance = db['instances'].find_one({'_id':instance_id})
    if current_instance['lower_cost'] == current_instance['solution_cost']:
        db['instances'].update_one({"_id" : instance_id},{"$set": 
                          {"closed": True}
                         })  
#     db['submissions'].insert_one(submission)
    db['submissions'].update_one({"instance_id":submission["instance_id"], "map_id":submission["map_id"],"algo_id":submission["algo_id"]}, 
                                 {"$set":
                                     {"lower_cost":submission['lower_cost'], 
                                      "solution_cost" : submission[ "solution_cost"],
                                      "date" : submission["date"],
                                      "best_lower" : submission["best_lower"],
                                      "best_solution" : submission["best_solution"]
                                     
                                     
                                     }
                                 }, upsert=True)


def verify_submission(db,submission,instance_id,algo_name,path):
    current_instance = db['instances'].find_one({'_id':instance_id})
    if(submission['solution_cost']!= None):
        if(current_instance['lower_cost'] != None):
            if(current_instance['lower_cost'] > submission['solution_cost']):
                print("submission:"+ str(submission['solution_cost']))
                print("current:"+ str(current_instance['lower_cost']))
                print("Inserting error: existing lower cost larger than submission solution cost")
    if(submission['lower_cost'] != None):
        if(current_instance['solution_cost'] != None):
            if(submission['lower_cost'] > current_instance['solution_cost']):
#                 print("")  
                print("submission:"+ str(submission['lower_cost']))
                print("current:"+ str(current_instance['solution_cost']))
                print("Inserting error: submission lower cost larger than existing solution cost")  
            
def valid_results(db,map_name):
    scen_type = ["even","random"]
    map_id = db["maps"].find_one({"map_name": map_name})['_id']
    for i in range(1, 26):
        for scen_t in scen_type:
            print("validing: "+ map_name + " "+scen_t + " "+ str(i))
            scen_data =  db['scenarios'].find_one({"map_id": map_id ,"type_id": i, "scen_type" :  scen_t})
            scen_id = scen_data['_id']
            number_agent = scen_data['instances']
            for k in range(1,number_agent):
                instance_id = db['instances'].find_one({"scen_id": scen_id ,"agents": k})['_id']
                current_instance = db['instances'].find_one({'_id':instance_id})
                if(current_instance['lower_cost'] != None and current_instance['solution_cost'] != None):
                    if(current_instance['lower_cost'] > current_instance['solution_cost'] ):
                        print('error: ' + str(i) + " " + scen_t + " "+str(k))
                        print('error: ' + str(current_instance['lower_cost']) +" "+str(current_instance['solution_cost']))
                
                
                
def insert_map_results(db,map_name):
    algo_id = db['algorithms'].find_one({"algo_name":"Lazy-CBS"})['_id']      
    algo_name = "Lazy-CBS"
    scen_type = ["even","random"]
    map_id = db["maps"].find_one({"map_name": map_name})['_id']
    for i in range(1, 26):
        for scen_t in scen_type:
            result_file = "./results/"+map_name+"/statistic/"+map_name+"-"+scen_t+"-"+str(i)+"-60"
            if( os.path.exists(result_file) == False):
                continue
            print(result_file)
            df1 = load_data(result_file)
            scen_data =  db['scenarios'].find_one({"map_id": map_id ,"type_id": i, "scen_type" :  scen_t})
            scen_id = scen_data['_id']
            number_agent = scen_data['instances']
            for index, row in df1.iterrows():
                if(row['num_agents'] > number_agent):
                    break;
                instance_id = db['instances'].find_one({"scen_id": scen_id ,"agents": row['num_agents']})['_id']
                submission= {
                        "map_id": map_id,
                        "instance_id" :  instance_id,
                        "algo_id": algo_id,
                        "lower_cost" : None,
                        "solution_cost": None,
                        "date": str(datetime.datetime.now().date()),
                        "best_lower": False,
                        "best_solution": False
                    }
                path = None
                if row['solution cost'] != -1:
                    path_results =  "./results/"+map_name+"/path/"+map_name+"-"+scen_t+"-"+str(i)+"-60/"+str(row['num_agents']) +"-path.txt"
                    path = get_path(path_results,row['num_agents'])
                    submission["solution_cost"] = row['solution cost']
                if row['lower_bound'] != -1:
                    submission["lower_cost"] = row['lower_bound']
#                 verify_submission(db,submission,instance_id,algo_name,path)
                insert_submission(db,submission,instance_id,algo_name,path)

def update_map_progress(db, map_name):
    map_id = db["maps"].find_one({"map_name": map_name})['_id']
    scen_type = ["even","random"]
    total_s = 0
    total_l = 0
    for i in range(1, 26):
        for scen_t in scen_type: 
            scen_id = db['scenarios'].find_one({"map_id": map_id ,"type_id": i, "scen_type" :  scen_t})['_id']
        
            num_s = db["instances"].count_documents({"scen_id" : scen_id, "closed" : True})
            num_l = db["instances"].count_documents({"scen_id" : scen_id, "solution_cost" : {"$ne":None}})
            total_s = total_s + num_s
            total_l = total_l + num_l
            db['scenarios'].update_one({"_id" : scen_id}, {"$set":{
                "instances_closed" : num_s,
                "instances_solved" : num_l
            }})
    db['maps'].update_one({"_id" : map_id}, {"$set":{
                "instances_closed" : total_s,
                "instances_solved" : total_l
            }})
    
    
def upload_map_results(db, map_name):
    insert_map_results(db,map_name)
    update_map_progress(db, map_name)
         
def update_algo_data(db):
    algorithms = db['algorithms'].find({})
    for algo in algorithms: 
        bb = db['submissions'].count_documents({"algo_id": algo['_id'], "best_lower": True})
        cc = db['submissions'].count_documents({"algo_id": algo['_id'], "best_solution": True})
        dd = db['submissions'].count_documents({"algo_id": algo['_id'], "$expr": {"$eq": ["$lower_cost", "$solution_cost"]}})
        ee = db['submissions'].count_documents({"algo_id": algo['_id'], "$expr": {"$ne": [ "$solution_cost", None]}})
        db['algorithms'].update_one({"_id" : algo['_id']},{"$set":{ "best_lower":bb, "best_solution":cc, "instances_closed":dd,"instances_solved":ee}})   
                    

In [9]:
url = "mongodb://localhost:27017/"
client = pymongo.MongoClient(url)
db = client['MAPF_V2']
print("Connect successfully")
finished_map=[]
folder = "./results"
sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
for map_name in sub_folders:
    if map_name in finished_map:
        print("Skip finished map: " + map_name)
    else:
#         valid_results(db,map_name)
        upload_map_results(db,map_name)

Connect successfully
./results/random-64-64-20/statistic/random-64-64-20-even-1-60
./results/random-64-64-20/statistic/random-64-64-20-random-1-60
./results/random-64-64-20/statistic/random-64-64-20-even-2-60
./results/random-64-64-20/statistic/random-64-64-20-random-2-60
./results/random-64-64-20/statistic/random-64-64-20-even-3-60
./results/random-64-64-20/statistic/random-64-64-20-random-3-60
./results/random-64-64-20/statistic/random-64-64-20-even-4-60
./results/random-64-64-20/statistic/random-64-64-20-random-4-60
./results/random-64-64-20/statistic/random-64-64-20-even-5-60
./results/random-64-64-20/statistic/random-64-64-20-random-5-60
./results/random-64-64-20/statistic/random-64-64-20-even-6-60
./results/random-64-64-20/statistic/random-64-64-20-random-6-60
./results/random-64-64-20/statistic/random-64-64-20-even-7-60
./results/random-64-64-20/statistic/random-64-64-20-random-7-60
./results/random-64-64-20/statistic/random-64-64-20-even-8-60
./results/random-64-64-20/statistic

./results/room-32-32-4/statistic/room-32-32-4-random-11-60
./results/room-32-32-4/statistic/room-32-32-4-even-12-60
./results/room-32-32-4/statistic/room-32-32-4-random-12-60
./results/room-32-32-4/statistic/room-32-32-4-even-13-60
./results/room-32-32-4/statistic/room-32-32-4-random-13-60
./results/room-32-32-4/statistic/room-32-32-4-even-14-60
./results/room-32-32-4/statistic/room-32-32-4-random-14-60
./results/room-32-32-4/statistic/room-32-32-4-even-15-60
./results/room-32-32-4/statistic/room-32-32-4-random-15-60
./results/room-32-32-4/statistic/room-32-32-4-even-16-60
./results/room-32-32-4/statistic/room-32-32-4-random-16-60
./results/room-32-32-4/statistic/room-32-32-4-even-17-60
./results/room-32-32-4/statistic/room-32-32-4-random-17-60
./results/room-32-32-4/statistic/room-32-32-4-even-18-60
./results/room-32-32-4/statistic/room-32-32-4-random-18-60
./results/room-32-32-4/statistic/room-32-32-4-even-19-60
./results/room-32-32-4/statistic/room-32-32-4-random-19-60
./results/roo

./results/empty-48-48/statistic/empty-48-48-random-12-60
./results/empty-48-48/statistic/empty-48-48-even-13-60
./results/empty-48-48/statistic/empty-48-48-random-13-60
./results/empty-48-48/statistic/empty-48-48-even-14-60
./results/empty-48-48/statistic/empty-48-48-random-14-60
./results/empty-48-48/statistic/empty-48-48-even-15-60
./results/empty-48-48/statistic/empty-48-48-random-15-60
./results/empty-48-48/statistic/empty-48-48-even-16-60
./results/empty-48-48/statistic/empty-48-48-random-16-60
./results/empty-48-48/statistic/empty-48-48-even-17-60
./results/empty-48-48/statistic/empty-48-48-random-17-60
./results/empty-48-48/statistic/empty-48-48-even-18-60
./results/empty-48-48/statistic/empty-48-48-random-18-60
./results/empty-48-48/statistic/empty-48-48-even-19-60
./results/empty-48-48/statistic/empty-48-48-random-19-60
./results/empty-48-48/statistic/empty-48-48-even-20-60
./results/empty-48-48/statistic/empty-48-48-random-20-60
./results/empty-48-48/statistic/empty-48-48-eve

./results/ost003d/statistic/ost003d-even-13-60
./results/ost003d/statistic/ost003d-random-13-60
./results/ost003d/statistic/ost003d-even-14-60
./results/ost003d/statistic/ost003d-random-14-60
./results/ost003d/statistic/ost003d-even-15-60
./results/ost003d/statistic/ost003d-random-15-60
./results/ost003d/statistic/ost003d-even-16-60
./results/ost003d/statistic/ost003d-random-16-60
./results/ost003d/statistic/ost003d-even-17-60
./results/ost003d/statistic/ost003d-random-17-60
./results/ost003d/statistic/ost003d-even-18-60
./results/ost003d/statistic/ost003d-random-18-60
./results/ost003d/statistic/ost003d-even-19-60
./results/ost003d/statistic/ost003d-random-19-60
./results/ost003d/statistic/ost003d-even-20-60
./results/ost003d/statistic/ost003d-random-20-60
./results/ost003d/statistic/ost003d-even-21-60
./results/ost003d/statistic/ost003d-random-21-60
./results/ost003d/statistic/ost003d-even-22-60
./results/ost003d/statistic/ost003d-random-22-60
./results/ost003d/statistic/ost003d-even

./results/ht_mansion_n/statistic/ht_mansion_n-random-18-60
./results/ht_mansion_n/statistic/ht_mansion_n-even-19-60
./results/ht_mansion_n/statistic/ht_mansion_n-random-19-60
./results/ht_mansion_n/statistic/ht_mansion_n-even-20-60
./results/ht_mansion_n/statistic/ht_mansion_n-random-20-60
./results/ht_mansion_n/statistic/ht_mansion_n-even-21-60
./results/ht_mansion_n/statistic/ht_mansion_n-random-21-60
./results/ht_mansion_n/statistic/ht_mansion_n-even-22-60
./results/ht_mansion_n/statistic/ht_mansion_n-random-22-60
./results/ht_mansion_n/statistic/ht_mansion_n-even-23-60
./results/ht_mansion_n/statistic/ht_mansion_n-random-23-60
./results/ht_mansion_n/statistic/ht_mansion_n-even-24-60
./results/ht_mansion_n/statistic/ht_mansion_n-random-24-60
./results/ht_mansion_n/statistic/ht_mansion_n-even-25-60
./results/ht_mansion_n/statistic/ht_mansion_n-random-25-60
./results/warehouse-10-20-10-2-1/statistic/warehouse-10-20-10-2-1-even-1-60
./results/warehouse-10-20-10-2-1/statistic/warehouse-

...

./results/room-64-64-8/statistic/room-64-64-8-random-16-60
./results/room-64-64-8/statistic/room-64-64-8-even-17-60
./results/room-64-64-8/statistic/room-64-64-8-random-17-60
./results/room-64-64-8/statistic/room-64-64-8-even-18-60
./results/room-64-64-8/statistic/room-64-64-8-random-18-60
./results/room-64-64-8/statistic/room-64-64-8-even-19-60
./results/room-64-64-8/statistic/room-64-64-8-random-19-60
./results/room-64-64-8/statistic/room-64-64-8-even-20-60
./results/room-64-64-8/statistic/room-64-64-8-random-20-60
./results/room-64-64-8/statistic/room-64-64-8-even-21-60
./results/room-64-64-8/statistic/room-64-64-8-random-21-60
./results/room-64-64-8/statistic/room-64-64-8-even-22-60
./results/room-64-64-8/statistic/room-64-64-8-random-22-60
./results/room-64-64-8/statistic/room-64-64-8-even-23-60
./results/room-64-64-8/statistic/room-64-64-8-random-23-60
./results/room-64-64-8/statistic/room-64-64-8-even-24-60
./results/room-64-64-8/statistic/room-64-64-8-random-24-60
./results/roo

./results/maze-128-128-10/statistic/maze-128-128-10-even-12-60
./results/maze-128-128-10/statistic/maze-128-128-10-random-12-60
./results/maze-128-128-10/statistic/maze-128-128-10-even-13-60
./results/maze-128-128-10/statistic/maze-128-128-10-random-13-60
./results/maze-128-128-10/statistic/maze-128-128-10-even-14-60
./results/maze-128-128-10/statistic/maze-128-128-10-random-14-60
./results/maze-128-128-10/statistic/maze-128-128-10-even-15-60
./results/maze-128-128-10/statistic/maze-128-128-10-random-15-60
./results/maze-128-128-10/statistic/maze-128-128-10-even-16-60
./results/maze-128-128-10/statistic/maze-128-128-10-random-16-60
./results/maze-128-128-10/statistic/maze-128-128-10-even-17-60
./results/maze-128-128-10/statistic/maze-128-128-10-random-17-60
./results/maze-128-128-10/statistic/maze-128-128-10-even-18-60
./results/maze-128-128-10/statistic/maze-128-128-10-random-18-60
./results/maze-128-128-10/statistic/maze-128-128-10-even-19-60
./results/maze-128-128-10/statistic/maze-

./results/empty-16-16/statistic/empty-16-16-random-24-60
./results/empty-16-16/statistic/empty-16-16-even-25-60
./results/empty-16-16/statistic/empty-16-16-random-25-60
./results/empty-32-32/statistic/empty-32-32-even-1-60
./results/empty-32-32/statistic/empty-32-32-random-1-60
./results/empty-32-32/statistic/empty-32-32-even-2-60
./results/empty-32-32/statistic/empty-32-32-random-2-60
./results/empty-32-32/statistic/empty-32-32-even-3-60
./results/empty-32-32/statistic/empty-32-32-random-3-60
./results/empty-32-32/statistic/empty-32-32-even-4-60
./results/empty-32-32/statistic/empty-32-32-random-4-60
./results/empty-32-32/statistic/empty-32-32-even-5-60
./results/empty-32-32/statistic/empty-32-32-random-5-60
./results/empty-32-32/statistic/empty-32-32-even-6-60
./results/empty-32-32/statistic/empty-32-32-random-6-60
./results/empty-32-32/statistic/empty-32-32-even-7-60
./results/empty-32-32/statistic/empty-32-32-random-7-60
./results/empty-32-32/statistic/empty-32-32-even-8-60
./resul

./results/maze-128-128-2/statistic/maze-128-128-2-even-19-60
./results/maze-128-128-2/statistic/maze-128-128-2-random-19-60
./results/maze-128-128-2/statistic/maze-128-128-2-even-20-60
./results/maze-128-128-2/statistic/maze-128-128-2-random-20-60
./results/maze-128-128-2/statistic/maze-128-128-2-even-21-60
./results/maze-128-128-2/statistic/maze-128-128-2-random-21-60
./results/maze-128-128-2/statistic/maze-128-128-2-even-22-60
./results/maze-128-128-2/statistic/maze-128-128-2-random-22-60
./results/maze-128-128-2/statistic/maze-128-128-2-even-23-60
./results/maze-128-128-2/statistic/maze-128-128-2-random-23-60
./results/maze-128-128-2/statistic/maze-128-128-2-even-24-60
./results/maze-128-128-2/statistic/maze-128-128-2-random-24-60
./results/maze-128-128-2/statistic/maze-128-128-2-even-25-60
./results/maze-128-128-2/statistic/maze-128-128-2-random-25-60
./results/warehouse-20-40-10-2-2/statistic/warehouse-20-40-10-2-2-even-1-60
./results/warehouse-20-40-10-2-2/statistic/warehouse-20-

./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-random-6-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-even-7-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-random-7-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-even-8-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-random-8-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-even-9-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-random-9-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-even-10-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-random-10-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-even-11-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-random-11-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-even-12-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-random-12-60
./results/lt_gallowstemplar_n/statistic/lt_gallowstemplar_n-even-13-60

In [10]:
url = "mongodb://localhost:27017/"
client = pymongo.MongoClient(url)
db = client['MAPF_V2']
print("Connect successfully")
update_algo_data(db)

Connect successfully
